# Playing against the intelligent agent

This notebook allow someone to test the graphic environnement and try de play against the trained agent.

IMPORTANT: To run this notebook you need csv file containing the Q-table. This can be done by running the python file main.py. It is initially configured to train the agent through 100.000 before saving the table, which will few minutes.

In [1]:
import pygame
from random import randint
from mastermind import Mastermind
from AI_agent import Sarsa
from main import open_table
import time
import sys

pygame.init()

print("Loading q_table... Please wait")
# Reading table previously learned
q_table_dict = open_table(file_name="learnt_table_SARSA_passed.csv")

# Creating an instance of an agent. Already trained (using table learned before)
RL = Sarsa(q_table=q_table_dict, is_qtable=True)

print("Q_table loaded, agent initialized")

# Creating environment instance
env = Mastermind(human=True)


def parse_combi(combi_string, act_table):
    action_num = int(combi_string[:len(combi_string)-2])
    return act_table[action_num]


def submit(value):
    try:
        a, b = zip(*[(a, b) for a, b in zip(env.combi, value) if a != b])
        a = list(a)
    except:
        return 4, 0
    for i in b:
        try:
            a.remove(i)
        except:
            continue
    return 4 - len(b), len(b) - len(a)


def play():
    ready = env.ready

    while True:
        for ev in pygame.event.get():
            if ev.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            elif ev.type == pygame.MOUSEBUTTONDOWN:
                if env.palet_rect.collidepoint(ev.pos) and len(env.combi) < 4:
                    x, y = ev.pos
                    x -= env.palet_rect.x
                    y -= env.palet_rect.y
                    try:
                        if env.palet_mask.get_at((x, y)):
                            env.combi.append(x // 30)
                            env.screen.blit(env.palette, (80 + (len(env.combi) - 1) * 30, 20),
                                            (x // 30 * 30, 0, 30, 30))
                            pygame.display.flip()
                    except:
                        continue
                elif env.reset_rect.collidepoint(ev.pos):
                    env.combi = []
                    env.screen.blit(env.mmbg, (0, 0))
                    env.screen.blit(env.palette, env.palet_rect)
                    pygame.display.flip()
                elif env.go_rect.collidepoint(ev.pos):
                    if len(env.combi) != 4:
                        env.combi = [randint(0, 7), randint(0, 7), randint(0, 7), randint(0, 7)]
                        for i in range(4):
                            env.screen.blit(env.palette, (80 + (i) * 30, 20), (env.combi[i] * 30, 0, 30, 30))
                            pygame.display.flip()
                            time.sleep(1)

                    ready = True
                    # Deploy AI agent function here

                    observation = "init"
                    for ligne in range(10):

                        action = RL.choose_action(observation)
                        ans = env.step(action, ligne)
                        prediction = parse_combi(ans[0], env.list_action)
                        feedback = env.feedback(prediction)
                        observation = str(action) + str(feedback[0]) + str(feedback[1])

                        for i in range(4):
                            r = env.screen.blit(env.palette, ((i + 1) * 30 + 20, ligne * 35 + 70),
                                                (prediction[i] * 30, 0, 30, 30))
                            pygame.display.update(r)
                        placed, misplaced = submit(prediction)
                        for e, c in enumerate([(255, 0, 0)] * placed + [(255, 255, 255)] * misplaced):
                            r = pygame.draw.circle(env.screen, c, (190 + e * 10, ligne * 35 + 85), 2, 0)
                            pygame.display.update(r)

                        if placed == 4:
                            break

                    if placed == 4:
                        pygame.display.set_caption('WON!')
                    else:
                        pygame.display.set_caption('LOST!')
                else:
                    break


play()
pygame.quit()
sys.exit()


pygame 2.5.2 (SDL 2.28.2, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html
Loading q_table... Please wait
